In [1]:
import pandas as pd 
import numpy as np 
import glob 
from pyspark.sql import SparkSession
import sqlite3
from langdetect import detect
from pyspark.sql.functions import when, lit, col, udf
from mrjob.job import MRJob
import re

In [2]:
spark = SparkSession.builder \
    .appName("Redit app")\
    .master("yarn")\
    .config("spark.driver.memory","32G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.3")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f792b69-f079-4308-88f6-a2e222409e13;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.3 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lomb

In [3]:
sc=spark.sparkContext.getConf().getAll()

In [4]:
#creating a spark section 
# spark = SparkSession.builder.master("local[1]").appName('Reddit app').getOrCreate()

In [5]:
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/dis_materials/data_reddit.csv")

In [6]:
#df.take(5)

In [7]:

# Create RDD from parallelize  
K=list(df.columns) 

print(K)

['created_utc', 'ups', 'subreddit_id', 'link_id', 'name', 'score_hidden', 'author_flair_css_class', 'author_flair_text', 'subreddit', 'id', 'removal_reason', 'gilded', 'downs', 'archived', 'author', 'score', 'retrieved_on', 'body', 'distinguished', 'edited', 'controversiality', 'parent_id']


In [8]:
for i in K:
    df[[i]].show()
    

+--------------------------------+
|                     created_utc|
+--------------------------------+
|                      1430438400|
|読みたいが買ったら負けな気がする|
|             図書館に出ねーかな"|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|            You and her simpl...|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|                      1430438400|
|            I shot my dog. I ...|
|            Well I was loadin...|
+--------------------------------+
only showing top 20 rows

+----+
| ups|
+----+
|   4|
|null|
|null|
|   4|
|   0|
|   3|
|   3|
|   1|
|   6|
|null|
|   2|
|   6|
|   5|
|   4|
|   1|
|   1|
|   3|
|  14|
|null|
|null

+----------+
| parent_id|
+----------+
|      null|
|      null|
|      null|
| t3_34g8mx|
|t1_cqufim0|
|t1_cqug2sr|
| t3_34fvry|
|t1_cqug10q|
|      null|
|      null|
| t3_34e7uq|
| t3_34gcwh|
| t3_34gmag|
| t3_34gmq6|
|t1_cqu4f8j|
|t1_cqs3tw2|
|t1_cqucws5|
|      null|
|      null|
|      null|
+----------+
only showing top 20 rows



In [9]:
df2 = df.dropDuplicates()
df2 = df2.na.drop(subset=["subreddit","subreddit_id","body"])

In [10]:
def isEnglish(text):
    try:
        if detect(text) != 'en':
            return "No"
        return "Yes"
    except:
        return "No"
    return "No"
# Convert function to UDF
isEnglishUDF = udf(lambda z: isEnglish(z))

In [11]:
# df_english = df2.withColumn("English",isEnglishUDF(col("body")))

In [12]:
df_english = df2.drop("created_utc","link_id","name","score_hidden","id","removal_reason","gilded","retrieved_on","edited","controversiality")

In [13]:
channels = df_english.select('subreddit').countApproxDistinct()

AttributeError: 'DataFrame' object has no attribute 'countApproxDistinct'

In [ ]:
channels 


In [ ]:
# import pyspark.sql.functions as F
# p=df_english[['subreddit']]
# p.coalesce(1).write.format("text").option("header", "false").mode("append").save("output.txt")

In [ ]:
channels = df_english.select('subreddit').distinct().rdd.map(lambda r:r[0]).collect()

In [ ]:
#channels 

In [ ]:
import re 
def chanels():
    pq=list(channels)
    p=[]
    for j, i in enumerate(pq):
        re.compile(r"[\w']+")
#         print(i)
        i=list(i)
        if len(i)<20:
                #print(i)
                #df=(df_english.filter(df_english.subreddit==str(i)))
                df=df_english.where(df_english.subreddit==str(i))
                chanel=df.toPandas()
                chanel.to_csv(r'pandas.txt', header=None, index=None, sep=' ', mode='a')
        if j>4:
            break



In [ ]:
chanels()n

In [ ]:
PPS=chanels()

In [ ]:
#df_english.select('subreddit').distinct().collect()
spark.sparkContext.parallelize(range(1000)).map(str).countApproxDistinct()

In [ ]:
    pq=list(channels)
    p=[]
    for j, i in enumerate(pq):
        re.compile(r"[\w']+")
#         print(i)
        if len(i)<20:
                i=list(i)
                df=(df_english.filter(df_english.subreddit==str(i)))
                chanel=df.toPandas()
                chanel.to_csv(r'pandas.txt', header=None, index=None, sep=' ', mode='a')
                #p.append(df)
        
        if j>2:
            break

In [ ]:

df_english.where(df_english.subreddit==str(anime).show())

In [ ]:
df_english.select('subreddit')

In [ ]:
rdd =spark.sparkContext.parallelize(df_english[[('subreddit')]].collect())

#spark.sparkContext.parallelize

In [ ]:
rdd.countByKey().items()


In [ ]:
pp=chanels() 

In [ ]:
rdd.countApproxDistinct()

In [ ]:
!python3 map_red_pup.py hdfs://namenode:9000/dis_materials/data_reddit.csv> output2.csv

In [ ]:
# for i in chanels():
#     print(type(i))
# #     print(i)

In [ ]:
sub=list(channels)
def subst():
    subs=[ ]
    for i in sub:
        if len(i)<10:
            subs.append(i)
    return subs 
rr=list(subst())

In [ ]:
#p=df_english[['subreddit']].show()
p=df_english[['subreddit']]

In [ ]:
K=list(p)

In [ ]:
p.show()

In [ ]:
countApproxDistinct()

In [ ]:
redit=list(rr)

In [ ]:
redit

In [ ]:
# from mapred import subreddit as rd 

# mr_job = rd()
# with mr_job.make_runner() as runner:
#     runner.run()
#     for line in runner.stream_output():
#         #handle each line however you like
#         print(line)


In [ ]:
# #p.agg(countDistinct('subreddit'))

# class subreddit(MRJob):
#     re.compile(r"[\w']+")
#     def mapper(self,channel,):
#         words = list(channel)
#         for word in words:
#             word.lower()
#             kb=list(word.split())
#             if len(kb)<5:
#                 yield kb ,1

#     def reducer(self, key, values):
#         return {key: sum(values)}
# if __name__ == '__main__':
#     subreddit.run()

In [ ]:
from mapred import subreddit as rd
!python3 mapred.py hdfs://namenode:9000/dis_materials/data_reddit.csv > chanel.csv  

In [ ]:
df_list = pd.DataFrame(redit)
# df.to_csv('filename.csv', index=False)

In [ ]:
pa=subreddit(df_list)

In [ ]:
rd.run()

In [ ]:
def mapper(channel,):
    re.compile(r"[\w']+")
    words = list(channel)
    cha=[]
    for word in words:
        word=word.lower()
        kb=list(word.split())
        if len(kb)<3:
            cha.append(kb)
    return cha
                
mapper(channels)

In [ ]:
def pp_chanel():
    #create a list of distinct chanels
    mem=sub
    #mem=list(df_english.select('subreddit').distinct().rdd.map(lambda r:r[0]).collect())
    for i in mem:
        yield i.upper(), 1

def reducer(subreddit,value):
    yield subreddit, value 
    
    

In [ ]:
!python3 mapred.py /home/ubuntu/chanel.csv> red.txt 


In [ ]:
p=pp_chanel()



In [ ]:
reducer(p,p)

In [ ]:

# #export PYSPARK_SUBMIT_ARGS="--master local[1] pyspark-shell"
# df = spark.read.format('jdbc').\
#      options(url='jdbc:sqlite:may2015.sqlite',\
#      dbtable='"hdfs://namenode:9000\dis_materials\atabase.sqlite"',driver='org.sqlite.JDBC').load()

In [ ]:
￼

# pd2=spark.read.csv("hdfs://namenode:9000/dis_materials/data_reddit.csv")
# pd2.show 

